In [11]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import numpy as np
import string
import random
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
import tensorflow as tf

In [6]:
def readGz(path):    
    for l in gzip.open(path, 'rt', encoding='cp437', errors='ignore'):        
        yield eval(l)

In [8]:
dataset = []
for l in readGz("goodreads_reviews_comics_graphic.json.gz"):    
    dataset.append(l)

In [9]:
dataset[0]

{'user_id': 'dc3763cdb9b2cae805882878eebb6a32',
 'book_id': '18471619',
 'review_id': '66b2ba840f9bd36d6d27f46136fe4772',
 'rating': 3,
 'review_text': 'Sherlock Holmes and the Vampires of London \n Release Date: April 2014 \n Publisher: Darkhorse Comics \n Story by: Sylvain Cordurie \n Art by: Laci \n Colors by: Axel Gonzabo \n Cover by: Jean Sebastien Rossbach \n ISDN: 9781616552664 \n MSRP: $17.99 Hardcover \n "Sherlock Holmes died fighting Professor Moriarty in the Reichenbach Falls. \n At least, that\'s what the press claims. \n However, Holmes is alive and well and taking advantage of his presumed death to travel the globe. \n Unfortunately, Holmes\'s plans are thwarted when a plague of vampirism haunts Britain. \n This book collects Sherlock Holmes and the Vampires of London Volumes 1 and 2, originally created by French publisher Soleil." - Darkhorse Comics \n When I received this copy of "Sherlock Holmes and the Vampires of London" I was Ecstatic! The cover art was awesome and 

In [15]:
train = dataset[:10000]
test = dataset[10000:20000]


In [23]:
trainX = [d["review_text"] for d in train]
trainY = [d["rating"] for d in train]
testX = [d["review_text"] for d in test]
testY = [d["rating"] for d in test]

In [21]:
def MSE(y, ypred):
    y = np.array(y)
    ypred = np.array(ypred)
    return np.mean((y-ypred)**2)

### Question #1A


In [22]:
tfidf1a = TfidfVectorizer(strip_accents="ascii", lowercase=True,ngram_range=(1,1),max_features=1000)
tfTrain = tfidf1a.fit_transform(trainX)   

In [25]:
tfTest = tfidf1a.transform(testX)

In [26]:
ridge1a = Ridge(alpha=1)

ridge1a.fit(tfTrain, trainY)
tfPredicted = ridge1a.predict(tfTest)

In [27]:
#MSE for 1a)
MSE(tfPredicted,testY)

1.1154806580750893

In [54]:
tokens1a = list(zip(ridge1a.coef_, tfidf1a.get_feature_names()))
tokens1a.sort() 
tokens1a.reverse()

In [56]:
# top 5 most positive tokens
tokens1a[:5]

[(1.9919107377226106, 'terrific'),
 (1.9125920850091802, 'superb'),
 (1.8583980663463953, 'great'),
 (1.6997278983885968, 'beautifully'),
 (1.6959122254731824, 'perfect')]

In [57]:
# top 5 most negative tokens
tokens1a[-5:]

[(-1.856407777698812, 'not'),
 (-1.903575592705677, 'says'),
 (-2.1254347938262055, 'disappointing'),
 (-2.4500875531114277, 'boring'),
 (-2.5918207654193153, 'worst')]

### Question #1B

In [28]:
tfidf1b = TfidfVectorizer(strip_accents="ascii", lowercase=True,ngram_range=(2,2),max_features=1000)
tfTrain1b = tfidf1b.fit_transform(trainX)   

In [30]:
tfTest1b = tfidf1b.transform(testX)

In [31]:
ridge1b = Ridge(alpha=1)

ridge1b.fit(tfTrain1b, trainY)
tfPredicted1b = ridge1b.predict(tfTest1b)

In [32]:
#MSE for 1b)
MSE(tfPredicted1b,testY)

1.2288547024077026

In [59]:
tokens1b = list(zip(ridge1b.coef_, tfidf1b.get_feature_names()))
tokens1b.sort() 
tokens1b.reverse()

In [60]:
# top 5 most positive tokens
tokens1b[:5]

[(1.389069251096654, 'loved this'),
 (1.355160602149556, 'continues to'),
 (1.3059432945295184, 'can wait'),
 (1.269640266166908, 'an excellent'),
 (1.2153650460590932, 'are also')]

In [61]:
# top 5 most negative tokens
tokens1b[-5:]

[(-1.3554697422590607, 'the problem'),
 (-1.3630487156326185, 'way too'),
 (-1.4959777906032645, 'unfortunately the'),
 (-1.6296080598746303, 'none of'),
 (-3.0096094600802368, 'the worst')]

### Question1C

In [33]:
tfidf1c = TfidfVectorizer(strip_accents="ascii", lowercase=True,ngram_range=(1,2),max_features=1000)
tfTrain1c = tfidf1c.fit_transform(trainX)   

In [34]:
tfTest1c = tfidf1c.transform(testX)

In [35]:
ridge1c = Ridge(alpha=1)

ridge1c.fit(tfTrain1c, trainY)
tfPredicted1c = ridge1c.predict(tfTest1c)

In [36]:
#MSE for 1b)
MSE(tfPredicted1c,testY)

1.129727049674162

In [62]:
tokens1c = list(zip(ridge1c.coef_, tfidf1c.get_feature_names()))
tokens1c.sort() 
tokens1c.reverse()

In [63]:
# top 5 most positive tokens
tokens1c[:5]

[(2.08884651457944, 'terrific'),
 (1.9650754411290603, 'great'),
 (1.7283047984878783, 'perfect'),
 (1.6998397297811227, 'excellent'),
 (1.6665890631121125, 'fantastic')]

In [64]:
# top 5 most negative tokens
tokens1c[-5:]

[(-1.4428888160622093, 'least'),
 (-1.5666543930579626, 'part of'),
 (-1.6189756425298727, 'not'),
 (-1.8605188293505934, 'unfortunately'),
 (-2.067707136895699, 'dull')]

### Question #2

In [45]:
def Cosine(x1,x2):
    numer = 0
    norm1 = 0
    norm2 = 0
    for a1,a2 in zip(x1,x2):
        numer += a1*a2
        norm1 += a1**2
        norm2 += a2**2
    if norm1*norm2:
        return numer / math.sqrt(norm1*norm2)
    return 0

In [78]:
query2 = tfidf1a.transform([trainX[0]]).toarray()[0]
test = tfidf1a.transform([trainX[1]]).toarray()[0]

In [79]:
Cosine(query2,test)

0.16679635334936602

In [82]:
similarities = []
for i in range(1,len(train)):
    temp = tfidf1a.transform([trainX[i]]).toarray()[0]
    cos = Cosine(query2, temp)
    similarities.append([cos,i])

In [85]:
similarities.sort(reverse=True)
similarities[:5]

[[0.6354899792038998, 8713],
 [0.6192151574342838, 1391],
 [0.6147157318637404, 8044],
 [0.6068424835251052, 8923],
 [0.6068424835251052, 8918]]

In [87]:
# ID of review
mostSim = similarities[0][1]
mostSim

8713

In [88]:
# highest cosine similarity to first review
trainX[mostSim]

'Originally posted on My Book Musings. \n *Copy provided by launch event organizer for an honest review. \n Anak Bathala: Kalem is the first book in a five-book graphic novel series, revolving around mythology of the old times. The first book is about Kalem\'s search for why he is called "Anak Bathala" (demi-god). The language in the book has a mixture of English and Filipino terms, with Baybayin and Surat Mangyan scripts written in some parts of the book. \n A brief overview of the whole series from their Facebook page: \n Anak Bathala is the epic adventure of Kalem that showcases rich Filipino mythology and culture. It exhibits native Filipino beliefs and folk work intertwined with values. The other books included in the series collection are: Yamal, Arau, Yesha and Anak Bathala. Each character embodies intrinsic Filipino traits like admirable virtues of courage, determination, self-discipline, prudence, camaraderie and leadership. Their stories also uncover the deceiving sphere of g

### Question #3

In [103]:
from fastFM import als
from scipy import sparse

In [98]:
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=5, l2_reg_w=0.1, l2_reg_V=0.5)

In [93]:
numInteractions = defaultdict()
for d in dataset:
    id = d['user_id']
    if id not in numInteractions.keys():
        numInteractions[id] = 1
    else:
        numInteractions[id] += 1

In [95]:
threeInteractions = []
for key in numInteractions.keys():
    if numInteractions[key] >= 3:
        threeInteractions.append(key)

In [97]:
threeInteractions = set(threeInteractions)

In [99]:
threeOrMoreDataset = []
for d in dataset:
    if d["user_id"] in threeInteractions:
        threeOrMoreDataset.append(d)

In [100]:
len(threeOrMoreDataset)

498312

### 3a)

In [102]:
userIndex = {} 
bookIndex = {}
for d in threeOrMoreDataset:
    if d['user_id'] not in userIndex: 
        userIndex[d['user_id']] = len(userIndex)
    if d['book_id'] not in bookIndex: 
        bookIndex[d['book_id']] = len(bookIndex)

In [104]:
X3a = sparse.lil_matrix((len(threeOrMoreDataset), len(userIndex) + len(bookIndex))) 
for i in range(len(threeOrMoreDataset)):
    user = userIndex[threeOrMoreDataset[i]['user_id']] 
    book = bookIndex[threeOrMoreDataset[i]['book_id']] 
    X3a[i, user] = 1
    X3a[i, len(userIndex) + book] = 1

In [105]:
Y3a = np.array([d['rating'] for d in threeOrMoreDataset])

In [106]:
# split data into train, test 50/50
# unable to figure out how to make test set w only last item 
X_train_3A = X3a[:len(threeOrMoreDataset)//2] 
X_test_3A = X3a[len(threeOrMoreDataset)//2:] 
y_train_3A = Y3a[:len(threeOrMoreDataset)//2] 
y_test_3A = Y3a[len(threeOrMoreDataset)//2:]

In [107]:
fm.fit(X_train_3A, y_train_3A) 
y_train_pred_3a = fm.predict(X_train_3A) 
y_test_pred_3a = fm.predict(X_test_3A)

In [108]:
print("Train MSE: {0} \n Test MSE: {1}".format(MSE(y_train_3A,y_train_pred_3a), MSE(y_test_3A,y_test_pred_3a)))

Train MSE: 0.16367998663387837 
 Test MSE: 1.2716459521917163


### 3B)

In [109]:
import dateutil

In [111]:
userIDs = {}
itemIDs = {}
interactions = []
interactionsPerUser = defaultdict(list)
for d in threeOrMoreDataset: 
    u = d['user_id']
    i = d['book_id']
    t = d['date_added'] 
    r = d['rating']
    dt = dateutil.parser.parse(t)
    t = int(dt.timestamp())
    if u not in userIDs: 
        userIDs[u] = len(userIDs) 
    if i not in itemIDs: 
        itemIDs[i] = len(itemIDs) 
    interactions.append((t, u, i, r)) 
    interactionsPerUser[u].append((t, i, r))

In [112]:
interactions.sort()


In [114]:
itemIDs['blank'] = len(itemIDs)

In [113]:
interactionsWithPrevious = []
for u in interactionsPerUser: 
    interactionsPerUser[u].sort()
    prevItem = 'blank'
    for (t, i, r) in interactionsPerUser[u]:
        interactionsWithPrevious.append((t, u, i, prevItem, r)) 
        prevItem = i

In [115]:
X3B = sparse.lil_matrix((len(threeOrMoreDataset), 2*len(itemIDs)))

In [116]:
y3b = np.array([])
for i in range(len(interactionsWithPrevious)):
    item = itemIDs[interactionsWithPrevious[i][2]] 
    j = itemIDs[interactionsWithPrevious[i][3]]
    r = interactionsWithPrevious[i][4]
    y3b = np.append(y3b, r)
    X3B[i, item] = 1
    X3B[i, len(itemIDs) + j] = 1

In [117]:
X_train_3b = X3B[:len(threeOrMoreDataset)//2] 
X_test_3b = X3B[len(threeOrMoreDataset)//2:] 
y_train_3b = y3b[:len(threeOrMoreDataset)//2] 
y_test_3b = y3b[len(threeOrMoreDataset)//2:]

In [118]:
fm.fit(X_train_3b, y_train_3b) 
y_train_pred_3b = fm.predict(X_train_3b) 
y_test_pred_3b = fm.predict(X_test_3b)

In [119]:
print("Train MSE: {0} \n Test MSE: {1}".format(MSE(y_train_3b,y_train_pred_3b), MSE(y_test_pred_3b,y_test_3b)))

Train MSE: 15.423579326844676 
 Test MSE: 15.519368302383407


### 3C

In [120]:
X3C = sparse.lil_matrix((len(threeOrMoreDataset), len(userIDs) + 2*len(itemIDs))) 
y3c = np.array([])
for i in range(len(interactionsWithPrevious)):
    u = userIDs[interactionsWithPrevious[i][1]] 
    item = itemIDs[interactionsWithPrevious[i][2]] 
    j = itemIDs[interactionsWithPrevious[i][3]]
    r = interactionsWithPrevious[i][4]
    y3c = np.append(y3c, r)
    X3C[i, u] = 1
    X3C[i, len(userIDs) + item] = 1
    X3C[i, len(userIDs) + len(itemIDs) + j] = 1

In [121]:
X_train_3c = X3C[:len(threeOrMoreDataset)//2] 
X_test_3c = X3C[len(threeOrMoreDataset)//2:] 
y_train_3c = y3c[:len(threeOrMoreDataset)//2] 
y_test_3c = y3c[len(threeOrMoreDataset)//2:]

In [122]:
fm.fit(X_train_3c, y_train_3c) 
y_train_pred_3c = fm.predict(X_train_3c) 
y_test_pred_3c = fm.predict(X_test_3c)

In [124]:
print("Train MSE: {0} \n Test MSE: {1}".format(MSE(y_train_3c,y_train_pred_3c), MSE(y_test_3c,y_test_pred_3c)))

Train MSE: 15.476811459910877 
 Test MSE: 15.545373062035623


### 4) 
Instead of using the date of the book was added, I will use the time it was read

In [125]:
readAtData = []
for d in threeOrMoreDataset:
    if len(d["read_at"]) > 0:
        readAtData.append(d)
len(readAtData)

436032

In [126]:
userIDs = {}
itemIDs = {}
interactions = []
interactionsPerUser = defaultdict(list)
for d in readAtData: 
    u = d['user_id']
    i = d['book_id']
    t = d['date_added'] 
    r = d['rating']
    dt = dateutil.parser.parse(t)
    t = int(dt.timestamp())
    if u not in userIDs: 
        userIDs[u] = len(userIDs) 
    if i not in itemIDs: 
        itemIDs[i] = len(itemIDs) 
    interactions.append((t, u, i, r)) 
    interactionsPerUser[u].append((t, i, r))

In [127]:
itemIDs["blank"] = len(itemIDs)

In [128]:
interactionsWithPrevious = []
for u in interactionsPerUser: 
    interactionsPerUser[u].sort()
    prevItem = 'blank'
    for (t, i, r) in interactionsPerUser[u]:
        interactionsWithPrevious.append((t, u, i, prevItem, r)) 
        prevItem = i

In [131]:
X4 = sparse.lil_matrix((len(readAtData), 2*len(itemIDs)))

In [132]:
y4 = np.array([])
for i in range(len(interactionsWithPrevious)):
    item = itemIDs[interactionsWithPrevious[i][2]] 
    j = itemIDs[interactionsWithPrevious[i][3]]
    r = interactionsWithPrevious[i][4]
    y4 = np.append(y4, r)
    X4[i, item] = 1
    X4[i, len(itemIDs) + j] = 1

In [133]:
X_train_4 = X4[:len(threeOrMoreDataset)//2] 
X_test_4 = X4[len(threeOrMoreDataset)//2:] 
y_train_4 = y4[:len(threeOrMoreDataset)//2] 
y_test_4 = y4[len(threeOrMoreDataset)//2:]

In [134]:
fm.fit(X_train_4, y_train_4) 
y_train_pred_3c = fm.predict(X_train_4) 
y_test_pred_3c = fm.predict(X_test_4)

In [135]:
print("Train MSE: {0} \n Test MSE: {1}".format(MSE(y_train_4,y_train_pred_3c), MSE(y_test_pred_3c,y_test_4)))

Train MSE: 15.491121563973316 
 Test MSE: 15.533281584157303


In [136]:
# not much improvement on MSE